In [1]:
import nltk
import spacy
from pathlib import Path
import pandas as pd
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import pickle
from collections import Counter
import math




nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000
                             
        
                             
    

In [ ]:
def read_novels(path=Path.cwd() / "p1-texts" / "novels"):
    data = []
    for file in path.glob("*.txt"):
        title_name = file.stem
        parts = title_name.split("-")

        if len(parts) >= 3:
            title = ' '.join(part.replace('_', ' ') for part in parts[:-2])
            author = parts[-2].replace('_', ' ')
            year = parts[-1]